# PSF Photometry in Crowded Fields with Photutils

##### This notebook exemplifies the use of photutils.psf to perform point spread function photometry in crowded fields artificially created assuming that stars have a known Gaussian model

## Create an artificial image

##### Let's create an artificial image using photutils.datasets functionalities

In [ ]:
from photutils.datasets import make_random_gaussians
from photutils.datasets import make_noise_image
from photutils.datasets import make_gaussian_sources

num_sources = 150
min_flux = 500
max_flux = 5000
min_xmean = 16
max_xmean = 240
sigma_psf = 2.0
starlist = make_random_gaussians(num_sources, [min_flux, max_flux],
                                 [min_xmean, max_xmean],
                                 [min_xmean, max_xmean],
                                 [sigma_psf, sigma_psf],
                                 [sigma_psf, sigma_psf],
                                 random_state=123)
shape = (256, 256)
image = (make_gaussian_sources(shape, starlist) +
         make_noise_image(shape, type='poisson', mean=2., random_state=123))
print(starlist)

## Initialize instances for the DAOPhotPSFPhotometry object

##### In order to initiliaze a DAOPhotPSFPhotometry object, we have to load some other objects which will be used to perform psf photometry. More precisely, the required parts are:  a fitter (e.g., from astropy.fitting), a source detection (e.g., DAOStarFinder, from photutils.detection), a grouping functionality (e.g., DAOGroup from photutils.psf), a background estimator (e.g. MedianBackground, from photutils.background), and a psf model (e.g., IntegratedGaussianPRF from photutils.psf).

In [ ]:
from photutils.detection import DAOStarFinder
from photutils.psf import DAOGroup
from photutils.psf import IntegratedGaussianPRF
from photutils.background import MedianBackground
from photutils.background import StdBackgroundRMS
from astropy.modeling.fitting import LevMarLSQFitter
from astropy.stats import gaussian_sigma_to_fwhm

bkgrms = StdBackgroundRMS(sigma=3.)

std = bkgrms(image)

daofind = DAOStarFinder(threshold=5.0*std,
                        fwhm=sigma_psf*gaussian_sigma_to_fwhm)

daogroup = DAOGroup(1.5*sigma_psf*gaussian_sigma_to_fwhm)

median_bkg = MedianBackground(sigma=3.)

psf_model = IntegratedGaussianPRF(sigma=sigma_psf)

fitter = LevMarLSQFitter()

## Perform photometry

##### To actually perform psf photometry, we need first to initialize an DAOPhotPSFPhotometry object by passing along the objects loaded in the previous cell. Similarly to DAOGroup and DAOStarFinder, DAOPhotPSFPhotometry class was designed such that its objetcs are callable, which means that one can call them in the same fashion as calling a function. This, in fact, will call the appropriate method to perform photometry. 

In [ ]:
from photutils.psf import DAOPhotPSFPhotometry

daophot_photometry = DAOPhotPSFPhotometry(find=daofind, group=daogroup,
                                          bkg=median_bkg, psf=psf_model,
                                          fitter=LevMarLSQFitter(),
                                          niters=2, fitshape=(11,11))
result_tab, residual_image = daophot_photometry(image=image)

## Plot original and residual images

In [ ]:
import numpy as np
%matplotlib inline
from matplotlib import rcParams
import matplotlib.pyplot as plt
rcParams['image.cmap'] = 'viridis'
rcParams['image.aspect'] = 1  # to get images with square pixels
rcParams['figure.figsize'] = (20,10)
rcParams['image.interpolation'] = 'nearest'
rcParams['image.origin'] = 'lower'

plt.imshow(image)
#for i, row in enumerate(starlist):
#    plt.annotate(str(i+1), (row['x_mean'], row['y_mean']), 
#                 (5, 5), textcoords='offset points', 
#                 color='w')
plt.title('Simulated data')
plt.xlabel('x-position (pixel units)')
plt.ylabel('y-position (pixel units)')

In [ ]:
plt.imshow(residual_image)
plt.title('Residual')
plt.xlabel('x-position (pixel units)')
plt.ylabel('y-position (pixel units)')

In [ ]:
print(result_tab)

## Photometry with fixed positions
##### Now, let's perform photometry for the case that star positions are held fixed and one is interested only in fitting the flux.

In [ ]:
psf_model.x_0.fixed = True
psf_model.y_0.fixed = True

In [ ]:
positions = starlist['x_mean', 'y_mean']
positions['x_mean'].name = 'x_0'
positions['y_mean'].name = 'y_0'

In [ ]:
daophot_photometry = DAOPhotPSFPhotometry(group=daogroup,
                                          bkg=median_bkg, psf=psf_model,
                                          fitter=LevMarLSQFitter(),
                                          fitshape=(11,11))
result_tab, residual_image = daophot_photometry(image=image, positions=positions)

In [ ]:
plt.imshow(residual_image)
plt.title('Residual')
plt.xlabel('x-position (pixel units)')
plt.ylabel('y-position (pixel units)')

In [ ]:
print(result_tab)